In [24]:
# default_exp chart.chart

In [12]:
# hide
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# export
import bokeh
from bokeh.plotting import output_file, show
from bokeh.models import Label
from chamomile.chart.style import Style
from bokeh.models.ranges import FactorRange

In [18]:
#export
class Chart:
    def __init__(self, x_axis_type='linear', y_axis_type='linear', x_range=None, plot_width: int=None, plot_height: int=None):
        if plot_width == None:
            self.plot_width = 800
        else:
            self.plot_width = plot_width
        if plot_height == None:
            self.plot_height = 500
        else:
            self.plot_height = plot_height

        self.style = Style(self)
        self.figure = self._initialize_figure(x_axis_type, y_axis_type)
        self.style._apply_settings('chart')
        self._subtitle_glyph = self._add_subtitle_to_figure()


    def _initialize_figure(self, x_axis_type, y_axis_type):
        x_range = None
        y_range = None

        if x_axis_type == 'categorical':
            x_range = []
        if y_axis_type == 'categorical':
            y_range = []
        if x_axis_type == 'categorical' or y_axis_type == 'categorical':
            figure = bokeh.plotting.figure(x_range = x_range, y_range=y_range,
                                       plot_width = self.plot_width, plot_height=self.plot_height)
        else:
            figure = bokeh.plotting.figure(x_axis_type = x_axis_type, y_axis_type=y_axis_type,
                                       plot_width = self.plot_width, plot_height=self.plot_height)
        return figure


    def mask(self):
        pass

    def set_notion(self, title=None, subtitle=None, x_axis_label=None, y_axis_label=None,source=None):
        if title:
            self.set_title(title)
        if subtitle:
            self.set_subtitle(subtitle)
        if x_axis_label:
            self.set_x_axis_label(x_axis_label)
        if y_axis_label:
            self.set_y_axis_label(y_axis_label)
        if source:
            self.set_source(source)
        return self

    def _add_source_to_figure(self):
        source_text = ""
        source_text_color = '#898989'
        source_font_size= '10px'
        _source = Label(
            x=self.style.plot_width * .9,
            y=0,
            x_units='screen',
            y_units='screen',
            level='overlay',
            text=source_text,
            text_color=source_text_color,
            text_font_size=source_font_size,
            text_align='right',
            name='data_source'
        )
        self.figure.add_layout(_source, 'below')
        return _source
    
    def _add_subtitle_to_figure(self, subtitle_text=None):
        if subtitle_text is None:
            subtitle_text = ""
        subtitle_settings = self.style._get_settings('subtitle')
        _subtitle_glyph = bokeh.models.Title(
            text = subtitle_text,
            align = subtitle_settings['subtitle_align'],
            text_color = subtitle_settings['subtitle_text_color'],

        )

        self.figure.add_layout(_subtitle_glyph, subtitle_settings['subtitle_location'])
        return _subtitle_glyph

    def set_x_axis_label(self, x_axis_label):
        return self

    def set_y_axis_label(self, y_axis_label):
        return self

    def set_subtitle(self, subtitle):
        self._subtitle_glyph.text = subtitle
        return self

    def set_source(self, source):
        self._source.text = source
        return self


    def set_title(self, title):
        """Set the chart title.
        Args:
            title (str): Title text.
        Returns:
            Current chart object
        """
        self.figure.title.text = title
        return self

    def show(self, format='html'):
            """Show the chart.
            Args:
                format (str):
                    - 'html': Output chart as HTML.
                        Renders faster and allows for interactivity.
                        Charts saved as HTML in a Jupyter notebooks
                        WILL NOT display on Github.
                        Logos will not display on HTML charts.
                        Recommended when drafting plots.
                    - 'png': Output chart as PNG.
                        Easy to copy+paste into slides.
                        Will render logos.
                        Recommended when the plot is in a finished state.
                    - 'svg': Output as SVG.
                    """
            self._set_toolbar_for_format(format)

            if format == 'html':
                return bokeh.io.show(self.figure)
            elif format == 'png':
                image = self._figure_to_png()
                # Need to re-enable this when logos are added back.
                # image = self.logo._add_logo_to_image(image)
                return display(image)
            elif format == 'svg':
                return self._show_svg()
